<a href="https://colab.research.google.com/github/imt2020532/English_Hindi_Translation/blob/main/Eng_Hind_Transla_20231004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import torch

In [22]:
!pip install transformers tqdm

In [23]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, AdamW

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
csv_file_path = '/content/drive/MyDrive/Eng_Hind_Transla_20231004/hindi_english_parallel.csv'

In [26]:
!pip install sentencepiece


In [27]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import re

# Define a custom dataset class
class CustomTranslationDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_seq_length):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        english_sentence = self.data.iloc[idx]['english']

        # if isinstance(english_sentence, float):
        #   english_sentence = str(english_sentence)
        # Convert to lowercase
        #english_sentence = english_sentence.lower()

        # Remove special characters using regular expressions
        #english_sentence = re.sub(r'[^a-zA-Z0-9\s]', '', english_sentence)

        hindi_translation = self.data.iloc[idx]['hindi']

        # Remove special characters using regular expressions
        #hindi_translation = re.sub(r'[^\u0900-\u097F\s]', '', hindi_translation)

        # Tokenize and encode the input and target sequences
        inputs = self.tokenizer.encode_plus(
            english_sentence,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            add_special_tokens=True
        )
        targets = self.tokenizer.encode_plus(
            hindi_translation,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            add_special_tokens=True
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'target_ids': targets['input_ids'].squeeze(),
            'target_attention_mask': targets['attention_mask'].squeeze()
        }

model_name = 'google/mt5-small'

# Specify the desired max sequence length
max_seq_length = 128  # Adjust as needed

batch_size = 10
learning_rate = 1e-4
n_epochs = 1


# Define the model name and fetch the tokenizer and model
model_name = 'google/mt5-small'
tokenizer = MT5Tokenizer.from_pretrained(model_name)

# Create an instance of the custom dataset
custom_dataset = CustomTranslationDataset(csv_file_path, tokenizer, max_seq_length)

In [28]:
custom_dataset.data

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default
...,...,...
1561836,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...
1561837,मैंने गौर किया है कि यह न केवल अपने महत्त्वपूर...,है। I note that this is a landmark meeting – n...
1561838,उन्होंने मेरे समक्ष जो प्रदर्शन किया उसमें से ...,है। In the presentations that they made before...
1561839,खाद्य और जल सुरक्षा; पर्यावरण की दृष्टि से वहन...,्त है। Issues such as food and water security;...


In [29]:
custom_dataset.data = custom_dataset.data.dropna(subset=['english', 'hindi'])

In [30]:
custom_dataset.data

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default
...,...,...
1561835,Members making oath/affirmation,शपथ लेने/प्रतिज्ञान करने वाले सदस्य
1561836,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...
1561837,मैंने गौर किया है कि यह न केवल अपने महत्त्वपूर...,है। I note that this is a landmark meeting – n...
1561838,उन्होंने मेरे समक्ष जो प्रदर्शन किया उसमें से ...,है। In the presentations that they made before...


In [31]:
custom_dataset.data['english'] = custom_dataset.data['english'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x).lower()))

    # Convert 'hindi' column to lowercase and remove special characters
custom_dataset.data['hindi'] = custom_dataset.data['hindi'].apply(lambda x: re.sub(r'[^\u0900-\u097F\s]', '', str(x)))

<ipython-input-31-94072857b416>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  custom_dataset.data['english'] = custom_dataset.data['english'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x).lower()))
<ipython-input-31-94072857b416>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  custom_dataset.data['hindi'] = custom_dataset.data['hindi'].apply(lambda x: re.sub(r'[^\u0900-\u097F\s]', '', str(x)))


In [32]:
custom_dataset.data

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,accerciser accessibility explorer
2,निचले पटल के लिए डिफोल्ट प्लगइन खाका,the default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,the default plugin layout for the top panel
4,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...,a list of plugins that are disabled by default
...,...,...
1561835,,
1561836,स्पष्टीकरणजहां इस उपधारा के अधीन हानि और लाभ ल...,139 ...
1561837,मैंने गौर किया है कि यह न केवल अपने महत्त्वपूर...,i note that this is a landmark meeting not o...
1561838,उन्होंने मेरे समक्ष जो प्रदर्शन किया उसमें से ...,in the presentations that they made before me...


In [33]:
from transformers import MT5ForConditionalGeneration

# Load the translation model
translation_model = MT5ForConditionalGeneration.from_pretrained(model_name)


In [34]:
import torch.optim as optim

# Define the loss function (e.g., cross-entropy)
criterion = torch.nn.CrossEntropyLoss()

# Define the optimizer (e.g., Adam)
optimizer = optim.Adam(translation_model.parameters(), lr=learning_rate)


In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
translation_model = translation_model.to(device)

In [36]:
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

In [37]:
import gc

In [38]:
torch.cuda.empty_cache()
gc.collect()

74

In [39]:
from tqdm import tqdm

In [40]:
for epoch in range(n_epochs):
    print("Epoch... ")
    print(epoch)
    # bar = tqdm(len(data_loader))
    for batch in tqdm(data_loader):
        #print(batch)
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)
        target_attention_mask = batch['target_attention_mask'].to(device)

        # Forward pass
        outputs = translation_model(
            input_ids=input_ids,
            labels = target_ids,
            attention_mask=attention_mask,
            decoder_input_ids=target_ids,
            decoder_attention_mask=target_attention_mask
        )

        # Calculate loss
        #loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), target_ids.view(-1))
        loss = outputs.loss
        # Backpropagation
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
        gc.collect()
        # bar.update()


Epoch... 
0


  0%|          | 0/155573 [00:00<?, ?it/s]


OutOfMemoryError: ignored

In [ ]:
# Define the path where you want to save the model
model_save_path = "/content/drive/MyDrive/Fine_Tuned_MT5_Model_20231005"

# Save the model and its tokenizer
translation_model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

# Load the model and tokenizer from the saved directory
loaded_model = MT5ForConditionalGeneration.from_pretrained(model_save_path)
loaded_tokenizer = MT5Tokenizer.from_pretrained(model_save_path)


In [ ]:
# Sample English sentence
english_sentence = "climb to the top"

# Tokenize the English sentence
input_ids = loaded_tokenizer.encode(english_sentence, return_tensors="pt", add_special_tokens=True)



In [ ]:
# Generate translation
translated_output = loaded_model.generate(
    input_ids,
    max_length=50,  # Adjust max length as needed
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
    early_stopping=True
)


In [ ]:
print(translated_output)

In [ ]:
# Decode the translation tokens using the same loaded tokenizer
translated_text = loaded_tokenizer.decode(translated_output[0], skip_special_tokens=True)


In [ ]:
print(translated_text)